In [93]:
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from keras_nlp.layers import TransformerEncoder, TransformerDecoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [101]:
class splitData():
    def __init__(self):
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []
    
    def split(self, x, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.1, random_state=42, shuffle=True)

In [102]:
features = ['district_encoder',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']

df = pd.read_excel('HCM_data.xlsx')
df_tranform = pd.DataFrame(data = Normalizer().fit_transform(\
    StandardScaler().fit_transform(df.loc[:, features].values)), columns = features)
y = df['price'].values
x = df_tranform[features].values
data = splitData()
data.split(x, y)

# RNN seq

## simple ANN


In [108]:

class ModelRNN(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = keras.layers.Dense(128, activation='relu') #fully connected networks
        self.dense2 = keras.layers.Dense(128, activation='relu')
        self.dense3 = keras.layers.Dense(128, activation='relu')
        self.dense4 = keras.layers.Dense(1, activation= 'relu')
        
        self.model = Sequential([self.dense1, self.dense2, self.dense3, self.dense4])
        
    def call(self,inputs):
        return self.model(inputs)
    
    def compile(self, optimizer="adam", loss="mse"):
        super().compile(optimizer, loss)
        
    def fit(self, data, epochs = 500, batch_size=128):
        super().fit(data.X_train, data.y_train, epochs, batch_size)
        
    def evaluation(self, data):
        y_pred = self.predict(data.X_test)
        
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)
        
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)

In [109]:
model_rnn = ModelRNN()
model_rnn.compile(optimizer='adam', loss='mse')
model_rnn.fit(data, epochs=10000000, batch_size =128)
model_rnn.evaluation(data)

Epoch 1/128
1/1 [==============================] - 0s 273ms/step - loss: 541.4588
Epoch 2/128
1/1 [==============================] - 0s 14ms/step - loss: 541.2914
Epoch 3/128
1/1 [==============================] - 0s 17ms/step - loss: 540.8826
Epoch 4/128
1/1 [==============================] - 0s 13ms/step - loss: 540.0930
Epoch 5/128
1/1 [==============================] - 0s 12ms/step - loss: 539.1042
Epoch 6/128
1/1 [==============================] - 0s 11ms/step - loss: 538.0129
Epoch 7/128
1/1 [==============================] - 0s 10ms/step - loss: 536.8526
Epoch 8/128
1/1 [==============================] - 0s 13ms/step - loss: 535.6166
Epoch 9/128
1/1 [==============================] - 0s 18ms/step - loss: 534.2833
Epoch 10/128
1/1 [==============================] - 0s 18ms/step - loss: 532.8297
Epoch 11/128
1/1 [==============================] - 0s 17ms/step - loss: 531.2378
Epoch 12/128
1/1 [==============================] - 0s 18ms/step - loss: 529.4854
Epoch 13/128
1/1 [======

# seq2seq

In [68]:
class Seq2Seq(keras.Model):
    
    def __init__(self):
        super().__init__()

        self.encoder = Sequential([
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
        ])

        self.decoder = Sequential([
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu')
        ])
        self.dense = Dense(1)
        
    def call(self, inputs):
        encoder_outputs = self.encoder(inputs)
        decoder_outputs = self.decoder(encoder_outputs)
        outputs = self.dense(decoder_outputs)
        return outputs
    
    def compile(self, optimizer='adam', loss='mse'):
        super().compile(optimizer, loss)
        
    def fit(self, x_train, y_train, epochs=1000, batch_size =128):
        super().fit(x_train, y_train, epochs, batch_size)
    
    def evaluation(self, data):
        y_pred = self.predict(data.X_test)
        
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)
        
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)
    




In [69]:
model = Seq2Seq()
model.compile(optimizer='adam', loss='mse')
model.fit(data.X_train, data.y_train, epochs=500, batch_size =128)
model.evaluation(data)

Epoch 1/128
9/9 [==============================] - 1s 3ms/step - loss: 527.4604
Epoch 2/128
9/9 [==============================] - 0s 4ms/step - loss: 444.6162
Epoch 3/128
9/9 [==============================] - 0s 4ms/step - loss: 432.5425
Epoch 4/128
9/9 [==============================] - 0s 3ms/step - loss: 418.3723
Epoch 5/128
9/9 [==============================] - 0s 3ms/step - loss: 407.1867
Epoch 6/128
9/9 [==============================] - 0s 3ms/step - loss: 398.5597
Epoch 7/128
9/9 [==============================] - 0s 3ms/step - loss: 401.4498
Epoch 8/128
9/9 [==============================] - 0s 4ms/step - loss: 395.7618
Epoch 9/128
9/9 [==============================] - 0s 3ms/step - loss: 392.2677
Epoch 10/128
9/9 [==============================] - 0s 4ms/step - loss: 390.8188
Epoch 11/128
9/9 [==============================] - 0s 4ms/step - loss: 386.2924
Epoch 12/128
9/9 [==============================] - 0s 4ms/step - loss: 384.7236
Epoch 13/128
9/9 [===================

In [90]:

class Transformer(keras.Model):
    
    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(Transformer, self).__init__()
        
        self.encoder = TransformerEncoder(num_layers, d_model, num_heads, dropout_rate)
        self.decoder = TransformerDecoder(num_layers, d_model, num_heads, dropout_rate)

    def call(self, inputs, training=False):
        encoder_outputs = self.encoder(inputs, training=training)
        decoder_outputs = self.decoder(encoder_outputs, training=training)
        
        return decoder_outputs

class TransformerEncoder_(keras.layers.Layer):
    
    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerEncoder, self).__init__()
        
        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                keras.layers.MultiHeadAttention(\
                    num_heads = num_heads,\
                    key_dim = d_model,\
                    dropout = dropout_rate\
                )\
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(
                keras.layers.Dense(\
                    d_model,\
                    activation = "relu"\
                )\
            )
            
    def call(self, inputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, inputs, training=training)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
            
        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
            
        return inputs

class TransformerDecoder_(keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dropout_rate):
        super(TransformerDecoder, self).__init__()
        
        self.attention_layers = []
        for _ in range(num_layers):
            self.attention_layers.append(
                keras.layers.MultiHeadAttention(
                    num_heads = num_heads,
                    key_dim = d_model,
                    dropout = dropout_rate
                )
            )
        self.feed_forward_layers = []
        for _ in range(num_layers):
            self.feed_forward_layers.append(\
                keras.layers.Dense(\
                    d_model,\
                    activation="relu"\
                )\
            )
            
        self.final_layer = keras.layers.Dense(
            1, # kích thước output
            activation="softmax" 
        )
        
    def call(self, inputs, encoder_outputs, training=False):
        for attention_layer in self.attention_layers:
            inputs = attention_layer(inputs, encoder_outputs, training=training)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        for feed_forward_layer in self.feed_forward_layers:
            inputs = feed_forward_layer(inputs)
            inputs = keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
        outputs = self.final_layer(inputs)
        return outputs

class TransformerModel(keras.Model):
    def __init__(self):
        super().__init__()
        
        self.encoder_input = Input(shape=(None, 9))
        
        self.encoder = TransformerEncoder(intermediate_dim=64, num_heads=8)
        
        self.encoder_output = encoder(encoder_input)
        
        self.decoder_input = Input(shape=(None, 9))
        self.decoder = TransformerDecoder(\
            intermediate_dim=64, num_heads=8
        )
        self.decoder_output = decoder(decoder_input, encoder_output)
        self.model = Model(self.encoder_input, self.encoder, self.decoder_output, self.decoder)
    
    def call(self, inputs):
        return self.model(inputs)
    
    def fit(self, x_train, y_train, epochs=1000, batch_size =128):
        super().fit(x_train, y_train, epochs, batch_size)
    2
    def evaluation(self, data):
        y_pred = self.predict(data.X_test)
        
        mse = mean_squared_error(data.y_test, y_pred)
        mae = mean_absolute_error(data.y_test, y_pred)
        evs = explained_variance_score(data.y_test, y_pred)
        
        print("MSE: ", mse)
        print("MAE: ", mae)
        print("variance: ", evs)

In [92]:
model_tran = TransformerModel()
model_tran.compile(optimizer='adam', loss='mse')
model_tran.fit(data, epochs=500, batch_size =128)
model_tran.evaluation(data)

NameError: name 'encoder' is not defined